In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup= BeautifulSoup(source,'lxml')
# print(soup.prettify())

Use Beatifulsoup liberary to get table from Wiki website

In [3]:
soup = BeautifulSoup(source,'lxml')
table = soup.find('div', class_="mw-parser-output").table
table_rows = table.find_all('tr')
l=[]
for tr in table_rows:
    td = tr.find_all('td')
#     print(td)
    row = [tr.text for tr in td]
#     print(row)
    l.append(row)
table_=pd.DataFrame(l, columns=["Postcode", "Borough","Neighborhood"])
df=table_
df.head(5)

,Postcode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


Start edit the table to get the target assignment

In [4]:
df_=df[df['Borough']!='Not assigned']
df_.drop(df_.index[0],inplace= True)
df_.reset_index(drop=True,inplace=True)
df_ = df_.replace('\n',',', regex=True)
df_['Neighborhood'] = np.where(df_['Neighborhood'] == "Not assigned,", df_['Borough']+",", df_['Neighborhood'])
df_.head(5)

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Postcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,"
3,M5A,Downtown Toronto,"Regent Park,"
4,M6A,North York,"Lawrence Heights,"


In [5]:
def insert_comma(x):
    y=x[:-1]
    return y   

Final step to get Datafarame and shape target

In [6]:
A=df_.groupby('Postcode')['Borough'].first()
B=df_.groupby('Postcode')['Neighborhood'].sum().apply(lambda x: insert_comma(x))
dataframe=pd.DataFrame([A,B]).transpose().reset_index()
dataframe.head(5)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
dataframe.shape

(103, 3)

In [8]:
# dataframe['Postcode'].head(5)

In [9]:
# ! pip install geocoder

In [10]:
# import geocoder # import geocoder

In [11]:
# dataframe['latitude']=None
# dataframe['longitude']=None

# for postal_code in dataframe['Postcode']:
# # initialize your variable to None
#     lat_lng_coords = None
# # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         print(postal_code)
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#         print('NA')
    
#     print(postal_code)

#     dataframe['latitude'].append(lat_lng_coords[0])
#     dataframe['latitude'].append(lat_lng_coords[1])
#doesnt work

Start read CSV file to get the Lat and Long for Neighborhood

In [12]:
df = pd.read_csv (r'C:\Users\HP\Desktop\projects\Coursera_Capstone\Geospatial_Coordinates.csv')
df.head(5)

FileNotFoundError: File b'C:\\Users\\HP\\Desktop\\projects\\Coursera_Capstone\\Geospatial_Coordinates.csv' does not exist

In [ ]:
# dataframe

Merge the two Dataframe to get the required target

In [ ]:
data_frame=pd.merge(df, dataframe, left_on='Postal Code', right_on='Postcode')
data_frame=data_frame[['Postcode','Borough','Neighborhood','Latitude','Longitude']]
data_frame.head(5)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Start the steps of clustering Toronto neighborhood as Newyork example

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_frame['Latitude'], data_frame['Longitude'], data_frame['Borough'], data_frame['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [ ]:
CLIENT_ID = 'SYNKAEL54LI1HUIAAZJ5VCZT3OISLVZSEAAASQDTUWRCXAVN' # your Foursquare ID
CLIENT_SECRET = 'JSJZWCJ3LNYT0YLLWXZKXXBFZ3JMJGYPX420H2AMWAQJF55K' # your Foursquare Secret
VERSION = '20140715' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
LIMIT=200
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
#         print(venues_list)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
 
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
#     print(nearby_venues.head(2))
    
    return(nearby_venues)

In [ ]:
Toronto_venues = getNearbyVenues(names=data_frame['Neighborhood'],latitudes=data_frame['Latitude'],longitudes=data_frame['Longitude'])

In [ ]:
Toronto_venues.shape

In [ ]:
Toronto_venues.head(5)

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")


# # add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 




In [ ]:
Toronto_onehot.head(5)
Toronto_onehot.columns.values
Toronto_onehot = Toronto_onehot[['Neighborhood' ,'Accessories Store', 'Afghan Restaurant', 'African Restaurant',
       'Airport', 'Airport Lounge', 'American Restaurant', 'Amphitheater',
       'Animal Shelter', 'Antique Shop', 'Aquarium', 'Art Gallery',
       'Art Museum', 'Arts & Crafts Store', 'Asian Restaurant',
       'Athletics & Sports', 'Auto Dealership', 'Auto Garage',
       'Automotive Shop', 'BBQ Joint', 'Baby Store', 'Badminton Court',
       'Bagel Shop', 'Bakery', 'Bank', 'Bar', 'Baseball Field',
       'Baseball Stadium', 'Basketball Stadium', 'Beach', 'Beach Bar',
       'Beer Bar', 'Beer Store', 'Belgian Restaurant', 'Bike Shop',
       'Bistro', 'Bookstore', 'Boutique', 'Bowling Alley',
       'Brazilian Restaurant', 'Breakfast Spot', 'Brewery', 'Bridal Shop',
       'Bridge', 'Bubble Tea Shop', 'Buffet', 'Building', 'Burger Joint',
       'Burrito Place', 'Bus Line', 'Bus Station', 'Bus Stop',
       'Business Service', 'Butcher', 'Cafeteria', 'Café',
       'Cajun / Creole Restaurant', 'Camera Store', 'Candy Store',
       'Cantonese Restaurant', 'Caribbean Restaurant', 'Castle',
       'Cemetery', 'Cheese Shop', 'Chinese Restaurant', 'Chiropractor',
       'Chocolate Shop', 'Church', 'Churrascaria', 'Climbing Gym',
       'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Gym',
       'College Lab', 'College Quad', 'College Rec Center',
       'College Stadium', 'College Theater', 'Comedy Club',
       'Comfort Food Restaurant', 'Comic Shop', 'Community Center',
       'Concert Hall', 'Construction & Landscaping', 'Convenience Store',
       'Cosmetics Shop', 'Coworking Space', 'Creperie',
       'Cuban Restaurant', 'Cupcake Shop', 'Curling Ice', 'Dance Studio',
       'Deli / Bodega', 'Department Store', 'Design Studio',
       'Dessert Shop', 'Dim Sum Restaurant', 'Diner', 'Discount Store',
       'Dive Bar', 'Dog Run', 'Doner Restaurant', 'Donut Shop',
       'Dumpling Restaurant', 'Eastern European Restaurant',
       'Electronics Store', 'Empanada Restaurant', 'Ethiopian Restaurant',
       'Event Space', 'Falafel Restaurant', 'Farm', 'Farmers Market',
       'Fast Food Restaurant', 'Field', 'Filipino Restaurant',
       'Fireworks Store', 'Fish & Chips Shop', 'Fish Market',
       'Flea Market', 'Flower Shop', 'Food', 'Food & Drink Shop',
       'Food Court', 'Food Truck', 'Fountain', 'French Restaurant',
       'Fried Chicken Joint', 'Frozen Yogurt Shop',
       'Fruit & Vegetable Store', 'Furniture / Home Store', 'Gaming Cafe',
       'Garden', 'Garden Center', 'Gas Station', 'Gastropub', 'Gay Bar',
       'General Entertainment', 'German Restaurant', 'Gift Shop',
       'Golf Course', 'Gourmet Shop', 'Greek Restaurant', 'Grocery Store',
       'Gym', 'Gym / Fitness Center', 'Gym Pool', 'Hakka Restaurant',
       'Halal Restaurant', 'Harbor / Marina', 'Hardware Store',
       'Hawaiian Restaurant', 'Health & Beauty Service',
       'Health Food Store', 'Historic Site', 'History Museum',
       'Hobby Shop', 'Hockey Arena', 'Home Service', 'Hookah Bar',
       'Hostel', 'Hot Dog Joint', 'Hotel', 'Hotel Bar',
       'Hotpot Restaurant', 'Housing Development', 'IT Services',
       'Ice Cream Shop', 'Indian Chinese Restaurant', 'Indian Restaurant',
       'Indie Movie Theater', 'Indie Theater', 'Indonesian Restaurant',
       'Intersection', 'Italian Restaurant', 'Japanese Restaurant',
       'Jazz Club', 'Jewelry Store', 'Jewish Restaurant', 'Juice Bar',
       'Karaoke Bar', 'Kids Store', 'Kitchen Supply Store',
       'Korean Restaurant', 'Lake', 'Latin American Restaurant',
       'Laundry Service', 'Light Rail Station', 'Lingerie Store',
       'Liquor Store', 'Lounge', 'Mac & Cheese Joint', 'Malay Restaurant',
       'Market', 'Martial Arts Dojo', 'Massage Studio', 'Mattress Store',
       'Mediterranean Restaurant', "Men's Store", 'Metro Station',
       'Mexican Restaurant', 'Middle Eastern Restaurant',
       'Miscellaneous Shop', 'Mobile Phone Shop',
       'Modern European Restaurant', 'Monument / Landmark',
       'Moroccan Restaurant', 'Motorcycle Shop', 'Movie Theater',
       'Museum', 'Music School', 'Music Store', 'Music Venue',
       'Nail Salon','New American Restaurant',
       'Nightclub', 'Noodle House', 'Office', 'Optical Shop',
       'Organic Grocery', 'Other Great Outdoors', 'Other Repair Shop',
       'Outdoors & Recreation', 'Pakistani Restaurant',
       'Paper / Office Supplies Store', 'Park', 'Pastry Shop',
       'Performing Arts Venue', 'Persian Restaurant', 'Pet Store',
       'Pharmacy', 'Photography Lab', 'Pide Place', 'Pie Shop',
       'Pilates Studio', 'Pizza Place', 'Playground', 'Plaza',
       'Poke Place', 'Polish Restaurant', 'Pool', 'Pool Hall',
       'Portuguese Restaurant', 'Poutine Place', 'Print Shop', 'Pub',
       'Ramen Restaurant', 'Record Shop', 'Recreation Center',
       'Rental Car Location', 'Residential Building (Apartment / Condo)',
       'Restaurant', 'River', 'Road', 'Rock Climbing Spot', 'Rock Club',
       'Roof Deck', 'Sake Bar', 'Salad Place', 'Salon / Barbershop',
       'Sandwich Place', 'Scenic Lookout', 'School', 'Sculpture Garden',
       'Seafood Restaurant', 'Shanghai Restaurant', 'Shoe Store',
       'Shop & Service', 'Shopping Mall', 'Skate Park', 'Skating Rink',
       'Ski Area', 'Ski Chalet', 'Smoke Shop', 'Smoothie Shop',
       'Snack Place', 'Soccer Field', 'Soccer Stadium', 'Social Club',
       'Soup Place', 'South American Restaurant',
       'Southern / Soul Food Restaurant', 'Souvlaki Shop', 'Spa',
       'Spanish Restaurant', 'Speakeasy', 'Sporting Goods Shop',
       'Sports Bar', 'Sports Club', 'Sri Lankan Restaurant',
       'Stationery Store', 'Steakhouse', 'Storage Facility', 'Street Art',
       'Supermarket', 'Supplement Shop', 'Sushi Restaurant', 'Taco Place',
       'Tailor Shop', 'Taiwanese Restaurant', 'Tanning Salon',
       'Tapas Restaurant', 'Tea Room', 'Tech Startup', 'Tennis Court',
       'Thai Restaurant', 'Theater', 'Theme Restaurant',
       'Thrift / Vintage Store', 'Tibetan Restaurant', 'Toy / Game Store',
       'Track', 'Trail', 'Train Station', 'Tree', 'Turkish Restaurant',
       'Udon Restaurant', 'University', 'Vegetarian / Vegan Restaurant',
       'Video Game Store', 'Video Store', 'Vietnamese Restaurant',
       'Warehouse Store', 'Waste Facility', 'Whisky Bar', 'Wine Bar',
       'Wine Shop', 'Wings Joint', "Women's Store", 'Yoga Studio', 'Zoo']]
# Toronto_onehot.head(5)

In [ ]:
Toronto_onehot.shape

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head(2)

In [ ]:
# num_top_venues = 5

# for hood in Toronto_grouped['Neighborhood']:
# #     print("----"+hood+"----")
#     temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
#     temp.columns = ['venue','freq']
#     temp = temp.iloc[1:]
#     temp['freq'] = temp['freq'].astype(float)
#     temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
kclusters = 8

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels',axis=1,inplace= True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = data_frame

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!
Toronto_merged_cluster=Toronto_merged.groupby('Cluster Labels')


In [ ]:
Toronto_merged.dropna(inplace=True)
Toronto_merged.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examin Clusters

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 7, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]